In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


model = ChatOpenAI(
    model=os.environ.get("MODEL_NAME"),
    temperature=0,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
)

In [2]:
response = model.invoke("离离原上草的下一句。")
response

AIMessage(content='“离离原上草”的下一句是：“一岁一枯荣。”\n\n这句诗出自唐代诗人白居易的《赋得古原草送别》。全诗如下：\n\n> 离离原上草，一岁一枯荣。  \n> 野火烧不尽，春风吹又生。  \n> 远芳侵古道，晴翠接荒城。  \n> 又送王孙去，萋萋满别情。\n\n其中，“离离”形容草木茂盛的样子，“一岁一枯荣”意为野草每年经历一次枯萎与繁茂的循环，展现了自然界的生生不息。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 17, 'total_tokens': 155, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-7de7a23a-2ef5-4787-89c8-e872de7bba1f', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--b35f72ea-c97f-47a8-8baf-df156fc3173a-0', usage_metadata={'input_tokens': 17, 'output_tokens': 138, 'total_tokens': 155, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [3]:
from langchain.messages import SystemMessage, HumanMessage

system_msg = SystemMessage("你是一个精通代码的助手。")

messages = [
    system_msg,
    HumanMessage("怎么写一个REST API？")
]

response = model.invoke(messages)
response

AIMessage(content='写一个 REST API 的步骤会根据你使用的编程语言和框架有所不同，但核心原则是一致的。下面我会以 **Python + Flask** 为例，展示如何快速搭建一个简单的 REST API。\n\n---\n\n## 🧱 一、REST API 基础概念\n\nREST（Representational State Transfer）是一种设计风格，通常使用 HTTP 方法来操作资源：\n\n- `GET`：获取资源\n- `POST`：创建资源\n- `PUT` / `PATCH`：更新资源\n- `DELETE`：删除资源\n\n---\n\n## 🐍 二、用 Python Flask 写一个简单 REST API\n\n### 1. 安装 Flask\n\n```bash\npip install flask\n```\n\n### 2. 编写代码（例如：管理用户）\n\n```python\nfrom flask import Flask, jsonify, request\n\napp = Flask(__name__)\n\n# 模拟数据库（实际项目中应使用数据库如 SQLite、PostgreSQL 等）\nusers = [\n    {"id": 1, "name": "Alice"},\n    {"id": 2, "name": "Bob"}\n]\n\n# GET /users - 获取所有用户\n@app.route(\'/users\', methods=[\'GET\'])\ndef get_users():\n    return jsonify(users)\n\n# GET /users/<id> - 获取单个用户\n@app.route(\'/users/<int:user_id>\', methods=[\'GET\'])\ndef get_user(user_id):\n    user = next((u for u in users if u["id"] == user_id), None)\n    if user:\n        return jsonify(user)\n    else:\n        return jsonify({"error": "User no

In [4]:
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    ...


model_with_tools = model.bind_tools([get_weather])
response = model_with_tools.invoke("What's the weather in Paris?")

response.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Paris'},
  'id': 'call_d7ba5a7dca52477ca6b62cd7',
  'type': 'tool_call'}]

In [5]:
chunks = []
full_message = None
for chunk in model.stream("Hi"):
    chunks.append(chunk)
    full_message = chunk if full_message is None else full_message + chunk

chunks

[AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--2a6e3789-1db9-417e-aa4a-d9987c6d9485'),
 AIMessageChunk(content='Hello', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--2a6e3789-1db9-417e-aa4a-d9987c6d9485'),
 AIMessageChunk(content='! How can', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--2a6e3789-1db9-417e-aa4a-d9987c6d9485'),
 AIMessageChunk(content=' I help', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--2a6e3789-1db9-417e-aa4a-d9987c6d9485'),
 AIMessageChunk(content=' you today?', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--2a6e3789-1db9-417e-aa4a-d9987c6d9485'),
 AIMessageChunk(content=' 😊', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--2a6e3789-1db9-417e-aa4a-d9987c6d9485'),
 AIMessageChunk(content='', additional_kwargs={}, response_metadat

In [6]:
full_message

AIMessageChunk(content='Hello! How can I help you today? 😊', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'stop', 'model_name': 'qwen3-max'}, id='lc_run--2a6e3789-1db9-417e-aa4a-d9987c6d9485', chunk_position='last')

In [9]:
from langchain.messages import AIMessage, ToolMessage


# After a model makes a tool call
ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "San Francisco"},
        "id": "call_123"
    }]
)

# Execute tool and create result message
weather_result = "Sunny, 72°F"
tool_message = ToolMessage(
    content=weather_result,
    tool_call_id="call_123"  # Must match the call ID
)

# Continue conversation
messages = [
    HumanMessage("What's the weather in San Francisco?"),
    ai_message,  # Model's tool call
    tool_message,  # Tool execution result
]
model.invoke(messages)  # Model processes the result

AIMessage(content='The weather in San Francisco is currently sunny with a temperature of 72°F.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-22b5d16c-4c32-4d3c-bbc2-5ac5feef0aed', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--59172e60-0c38-4b63-848f-d9b85398a62c-0', usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})